# Guest House - Hard

In [1]:
import datetime

import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@192.168.31.31:15432/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 100)

········


In [2]:
booking = pd.read_sql_table('booking', engine)
guest = pd.read_sql_table('guest', engine)
room = pd.read_sql_table('room', engine)
rate = pd.read_sql_table('rate', engine)
extra = pd.read_sql_table('extra', engine)

## 11.
Coincidence. Have two guests with the same surname ever stayed in the hotel on the evening? Show the last name and both first names. Do not include duplicates.

```
+-----------+------------+-------------+
| last_name | first_name | first_name  |
+-----------+------------+-------------+
| Davies    | Philip     | David T. C. |
| Evans     | Graham     | Mr Nigel    |
| Howarth   | Mr George  | Sir Gerald  |
| Jones     | Susan Elan | Mr Marcus   |
| Lewis     | Clive      | Dr Julian   |
| McDonnell | John       | Dr Alasdair |
+-----------+------------+-------------+
```

In [3]:
t = (guest.merge(booking, left_on='id', right_on='guest_id')
     [['last_name', 'first_name', 'booking_date', 'nights', 'id']])
a = t.merge(t, on='last_name', how='outer').query('first_name_x != first_name_y')
(a.loc[(((a['booking_date_x'] >= a['booking_date_y']) &
         (a['booking_date_x'] <= a['booking_date_y'] + 
          pd.to_timedelta(a['nights_y']-1, unit='D'))) |
        ((a['booking_date_y'] >= a['booking_date_x']) &
         (a['booking_date_y'] <= a['booking_date_x'] + 
          pd.to_timedelta(a['nights_x']-1, unit='D')))) &
       (a['id_x'] >= a['id_y']), :]
 [['last_name', 'first_name_y', 'first_name_x']]
 .sort_values('last_name')
 .drop_duplicates())

,last_name,first_name_y,first_name_x
387,Davies,Philip,David T. C.
299,Evans,Graham,Mr Nigel
253,Howarth,Mr George,Sir Gerald
102,Jones,Susan Elan,Mr Marcus
37,Lewis,Clive,Dr Julian
148,McDonnell,John,Dr Alasdair


## 12.
Check out per floor. The first digit of the room number indicates the floor – e.g. room 201 is on the 2nd floor. For each day of the week beginning 2016-11-14 show how many rooms are being vacated that day by floor number. Show all days in the correct order.

```
+------------+-----+-----+-----+
| i          | 1st | 2nd | 3rd |
+------------+-----+-----+-----+
| 2016-11-14 |   5 |   3 |   4 |
| 2016-11-15 |   6 |   4 |   1 |
| 2016-11-16 |   2 |   2 |   4 |
| 2016-11-17 |   5 |   3 |   6 |
| 2016-11-18 |   2 |   3 |   2 |
| 2016-11-19 |   5 |   5 |   1 |
| 2016-11-20 |   2 |   2 |   2 |
+------------+-----+-----+-----+
```

In [4]:
a = booking.assign(checkout_date=booking['booking_date'] + 
                     pd.to_timedelta(booking['nights'], unit='D'),
                   floor=booking['room_no'].astype(str).str.slice(0, 1))
a = (a.loc[a['checkout_date'].between('2016-11-14', '2016-11-20')])
a.replace({'floor': {'1': '1st', '2': '2nd', '3': '3rd', '4': '4th'}},
          inplace=True)
(a.groupby(['checkout_date', 'floor'])['room_no'].count().reset_index()
 .pivot(index='checkout_date', columns='floor'))

room_no        
floor             1st 2nd 3rd
checkout_date                
2016-11-14          5   3   4
2016-11-15          6   4   1
2016-11-16          2   2   4
2016-11-17          5   3   6
2016-11-18          2   3   2
2016-11-19          5   5   1
2016-11-20          2   2   2

## 13.
Free rooms? List the rooms that are free on the day 25th Nov 2016.

```
+-----+
| id  |
+-----+
| 207 |
| 210 |
| 304 |
+-----+
```

In [5]:
(room.loc[~room['id'].isin(
    booking.loc[(booking['booking_date'] <= '2016-11-25') & 
                (booking['booking_date']+pd.to_timedelta(booking['nights']-1, unit='D')
                 >= '2016-11-25'), 'room_no'])]
 [['id']])

,id
16,207
19,210
23,304


## 14.
Single room for three nights required. A customer wants a single room for three consecutive nights. Find the first available date in December 2016.

```
+-----+------------+
| id  | MIN(i)     |
+-----+------------+
| 201 | 2016-12-11 |
+-----+------------+
```

In [6]:
a = booking.assign(checkout_date = booking['booking_date'] + 
                   pd.to_timedelta(booking['nights'], unit='D'))
a = a.merge(room.loc[room['room_type']=='single'], 
            left_on='room_no', right_on='id', how='right')

a = (a.loc[((a['checkout_date'] >= '2016-12-01') & 
            (a['booking_date'] <= '2016-12-31')) |
           (a['checkout_date'] is None)]
     .sort_values(['room_no', 'booking_date'])
     [['booking_id', 'room_no', 'booking_date', 'nights', 'checkout_date']])
a['next_booking'] = (a.groupby(['room_no'])['booking_date']
                     .shift(-1).fillna('2017-01-01'))
a['diff'] = (a['next_booking']-a['checkout_date']).dt.days.fillna(31)

a.loc[(a['diff']>=3), ['room_no', 'checkout_date']].sort_values('checkout_date').iloc[:1]

,room_no,checkout_date
23,201,2016-12-11


## 15.
Gross income by week. Money is collected from guests when they leave. For each Thursday in November and December 2016, show the total amount of money collected from the previous Friday to that day, inclusive.

```
+------------+---------------+
| Thursday   | weekly_income |
+------------+---------------+
| 2016-11-03 |          0.00 |
| 2016-11-10 |      12608.94 |
| 2016-11-17 |      13552.56 |
| 2016-11-24 |      12929.69 |
| 2016-12-01 |      11685.14 |
| 2016-12-08 |      13093.79 |
| 2016-12-15 |       8975.87 |
| 2016-12-22 |       1395.77 |
| 2016-12-29 |          0.00 |
| 2017-01-05 |          0.00 |
+------------+---------------+
```

In [7]:
# generate Thursdays
sdate = datetime.date(2016, 11, 1)
edate = datetime.date(2016, 12, 31)
days_to_thu = (3 - sdate.weekday()) % 7
week_diff = ((edate - sdate).days - days_to_thu) // 7
thur = pd.DataFrame(
    {'Thursday': [sdate + datetime.timedelta(days=days_to_thu + 7 * more_wks) 
                  for more_wks in range(week_diff + 1)]})
thur['Thursday'] = thur['Thursday'].astype('datetime64[ns]')

In [8]:
a = (booking.merge(rate, left_on=['occupants', 'room_type_requested'],
                   right_on=['occupancy', 'room_type'], how='left'))
a['income'] = a['amount'].fillna(0) * a['nights'].fillna(0)
b = (booking.merge(extra[['booking_id', 'amount']], 
                   on='booking_id', how='left'))
b['income'] = b['amount'].fillna(0)

a = pd.concat([a[['booking_date', 'nights', 'income']],
               b[['booking_date', 'nights', 'income']]])
a['checkout_date'] = a['booking_date'] + \
    pd.to_timedelta(a['nights'], unit='D')
a['wkd'] = a['checkout_date'].dt.weekday

# weekday <= 3
cond1 = a['checkout_date'].dt.weekday <= 3
a.loc[cond1, 'Thursday'] = a.loc[cond1, 'checkout_date'] + \
    (pd.to_timedelta(3 - a.loc[cond1, 'wkd'], unit='D'))
# weekday >=4
cond2 = a['checkout_date'].dt.weekday > 3
a.loc[cond2, 'Thursday'] = a.loc[cond2, 'checkout_date'] + \
    (pd.to_timedelta(10 - a.loc[cond2, 'wkd'], unit='D'))

# a
a.merge(thur, on='Thursday', how='right').groupby('Thursday')[['income']].sum()

,income
Thursday,
2016-11-03,0.00
2016-11-10,12608.94
2016-11-17,13552.56
2016-11-24,12929.69
2016-12-01,11685.14
2016-12-08,13093.79
2016-12-15,8975.87
2016-12-22,1395.77
2016-12-29,0.00
